In [3]:
# Visualization_2 – Lesson 1 – Data Breaches (Connecting data, INPUT verze)
# -------------------------------------------------------------
import os, re, glob, importlib.util
import pandas as pd
import numpy as np

def is_colab():
    return importlib.util.find_spec("google.colab") is not None

def upload_via_colab():
    # otevře upload dialog a vrátí dočasnou cestu k prvnímu souboru
    from google.colab import files
    up = files.upload()  # uživatel vybere CSV/XLSX
    if not up:
        raise RuntimeError("Nebyl vybrán žádný soubor.")
    fname = list(up.keys())[0]
    print(f"[INFO] Nahráno: {fname}")
    return fname

# === 0) Získání cesty ze vstupu ===
print("Zadej úplnou nebo relativní cestu k souboru (.csv / .xlsx).")
print("Příklad:  C:/Users/ja/Downloads/data_breaches.csv  nebo  ./Data_Breaches/data_breaches.xlsx")
user_path = input("Cesta k souboru (ENTER = v Colabu otevřít upload dialog): ").strip()

if user_path == "":
    if is_colab():
        path = upload_via_colab()
    else:
        raise FileNotFoundError(
            "Nebyla zadána cesta a neběžíš v Google Colab.\n"
            "Prosím zadej cestu k .csv/.xlsx souboru do inputu a spusť znovu."
        )
else:
    # rozšíření zkratek a ověření existence
    path = os.path.expanduser(user_path)
    if not os.path.isfile(path):
        # drobná pomoc: pokud uživatel dal jen složku, zkus najít první csv/xlsx uvnitř
        if os.path.isdir(path):
            cand = glob.glob(os.path.join(path, "*.csv")) + glob.glob(os.path.join(path, "*.xlsx"))
            if cand:
                path = cand[0]
                print(f"[INFO] Ve složce nalezen soubor: {os.path.basename(path)}")
            else:
                raise FileNotFoundError("Ve zadané složce jsem nenašel žádné .csv/.xlsx soubory.")
        else:
            raise FileNotFoundError(f"Soubor {path!r} neexistuje.")

# === 1) Načtení dat (podpora CSV/Excel) ===
ext = os.path.splitext(path)[1].lower()
if ext == ".csv":
    try:
        data_breaches = pd.read_csv(path, sep=None, engine="python")
    except UnicodeDecodeError:
        data_breaches = pd.read_csv(path, sep=None, engine="python", encoding="utf-8", encoding_errors="ignore")
elif ext in (".xlsx", ".xls"):
    try:
        xls = pd.ExcelFile(path)
        # zvol list s nejvíc řádky
        best_sheet, max_rows = None, -1
        for s in xls.sheet_names:
            n = pd.read_excel(path, sheet_name=s, nrows=5).shape[0]
            if n > max_rows:
                best_sheet, max_rows = s, n
        data_breaches = pd.read_excel(path, sheet_name=best_sheet)
        print(f"[INFO] Excel sheet: {best_sheet}")
    except Exception as e:
        raise RuntimeError(f"Načtení Excelu selhalo: {e}")
else:
    raise ValueError("Podporovány jsou pouze .csv a .xlsx/.xls")

orig_df = data_breaches.copy()

# === 2) Základní profil ===
n_rows, n_cols = data_breaches.shape
print(f"\n[INFO] Načten soubor: {os.path.basename(path)} | tvar: {n_rows}×{n_cols}")
print("[INFO] Dtypes:")
print(data_breaches.dtypes)

# === 3) Heuristiky pro konverze ===
date_like, numeric_like = [], []
for col in data_breaches.columns:
    cl = col.lower()
    if re.search(r"(date|time|year)", cl):
        date_like.append(col)
    if re.search(r"(record|count|affected|size|num|amount|total)", cl):
        numeric_like.append(col)

changes = []

# Datumy → datetime
for col in date_like:
    if col in data_breaches.columns:
        before_na = data_breaches[col].isna().sum()
        data_breaches[col] = pd.to_datetime(data_breaches[col], errors="coerce")
        after_na = data_breaches[col].isna().sum()
        changes.append(("to_datetime", col, f"NaN změna: {after_na - before_na}"))

# Čísla z textu → float
def to_number(v):
    if pd.isna(v): return np.nan
    s = re.sub(r"[^\d\.\-]", "", str(v))
    if s in ("", "-", ".", "-.", ".-"): return np.nan
    try: return float(s)
    except: return np.nan

for col in numeric_like:
    if col in data_breaches.columns:
        before = data_breaches[col].notna().sum()
        data_breaches[col] = data_breaches[col].apply(to_number)
        after = data_breaches[col].notna().sum()
        changes.append(("to_numeric", col, f"ne-NaN: {before} → {after}"))

# Kategorie (text s malou kardinalitou)
cat_candidates = []
for col in data_breaches.select_dtypes(include=["object"]).columns:
    nun = data_breaches[col].nunique(dropna=True)
    if 1 < nun <= max(20, int(0.05 * len(data_breaches))):
        cat_candidates.append(col)
for col in cat_candidates:
    data_breaches[col] = data_breaches[col].astype("category")
    changes.append(("to_category", col, f"nunique={data_breaches[col].nunique()}"))

# === 4) Chybějící hodnoty ===
missing_summary = data_breaches.isna().sum().sort_values(ascending=False)

# === 5) Uložení očištěných dat vedle zdroje ===
out_dir = os.path.dirname(path) or "."
clean_path = os.path.join(out_dir, "data_breaches_clean.csv")
data_breaches.to_csv(clean_path, index=False)

# === 6) SOUHRNNÝ VÝPIS (povinná hláška) ===
print("\n================ SOUHRNNÝ VÝPIS =================")
print("co všechno bylo provedeno:")
print(f"- Soubor načten z: {path}")
print(f"- Původní tvar: {orig_df.shape[0]} řádků × {orig_df.shape[1]} sloupců")
print(f"- Aktuální tvar: {data_breaches.shape[0]} řádků × {data_breaches.shape[1]} sloupců")
print(f"- Kandidáti na datum: {date_like}")
print(f"- Kandidáti na čísla: {numeric_like}")
print(f"- Kandidáti na kategorie: {cat_candidates}")
print("- Provedené konverze/akce:")
for k, c, note in changes:
    print(f"  • {k:>12} :: {c} — {note}")
print("\n[INFO] Top 15 sloupců s chybějícími hodnotami:")
print(missing_summary.head(15))
print(f"\n[OK] Očištěný soubor uložen: {clean_path}")

Zadej úplnou nebo relativní cestu k souboru (.csv / .xlsx).
Příklad:  C:/Users/ja/Downloads/data_breaches.csv  nebo  ./Data_Breaches/data_breaches.xlsx
Cesta k souboru (ENTER = v Colabu otevřít upload dialog): C:\Users\Kulrychova\Documents\2. CodersLab-Data visualization_GitHub\data_breaches.csv


FileNotFoundError: Soubor 'C:\\Users\\Kulrychova\\Documents\\2. CodersLab-Data visualization_GitHub\\data_breaches.csv' neexistuje.

In [ ]:
# Visualization_2 – Lesson 1 – Data Breaches (INPUT + robustní nalezení cesty)
import os, re, glob, importlib.util
import pandas as pd
import numpy as np

def is_colab():
    return importlib.util.find_spec("google.colab") is not None

def upload_via_colab():
    from google.colab import files
    up = files.upload()
    if not up:
        raise RuntimeError("Nebyl vybrán žádný soubor.")
    fname = list(up.keys())[0]
    print(f"[INFO] Nahráno: {fname}")
    return fname

def normalize_path(p: str) -> str:
    # ořez uvozovek a mezer, převod zpětných lomítek
    p = p.strip().strip('"').strip("'")
    p = p.replace("\\\\", "\\")
    p = p.replace("\\", "/")
    p = os.path.expanduser(p)
    p = os.path.normpath(p)
    return p

def try_find_similar(target_path: str) -> str | None:
    """
    Pokud soubor neexistuje, podívej se do složky target_path (nebo do jejího rodiče),
    a zkus najít .csv/.xlsx se jménem podobným 'breach' nebo stejné jméno bez ohledu na velikost písmen.
    """
    base = normalize_path(target_path)
    folder, base_name = os.path.dirname(base), os.path.basename(base)
    name_lower = base_name.lower()

    # pokud folder neexistuje, zkus jeho rodiče (uživatel se mohl splést v části cesty)
    candidates_dirs = []
    if os.path.isdir(folder):
        candidates_dirs.append(folder)
    parent = os.path.dirname(folder)
    if os.path.isdir(parent):
        candidates_dirs.append(parent)

    for d in candidates_dirs:
        files_here = os.listdir(d)
        # 1) přesná shoda názvu bez ohledu na velikost písmen
        for f in files_here:
            if f.lower() == name_lower and os.path.isfile(os.path.join(d, f)):
                return os.path.join(d, f)
        # 2) CSV/XLSX obsahující 'breach' v názvu
        prio = [f for f in files_here if re.search(r"breach", f, re.I) and re.search(r"\.(csv|xlsx)$", f, re.I)]
        if prio:
            # vem první dle abecedy
            prio.sort()
            print(f"[INFO] Nalezen podobný soubor ve složce: {prio[0]}")
            return os.path.join(d, prio[0])
        # 3) jakýkoliv CSV/XLSX – fallback
        any_tab = [f for f in files_here if re.search(r"\.(csv|xlsx)$", f, re.I)]
        if any_tab:
            any_tab.sort()
            print(f"[INFO] Ve složce nalezen CSV/XLSX, vybírám: {any_tab[0]}")
            return os.path.join(d, any_tab[0])

    return None

print("Zadej úplnou nebo relativní cestu k souboru (.csv / .xlsx).")
print("Příklad:  C:/Users/ja/Downloads/data_breaches.csv  nebo  ./Data_Breaches/data_breaches.xlsx")
user_path = input("Cesta k souboru (ENTER = v Colabu otevřít upload dialog): ").strip()

# === Rozhodnutí, odkud brát data ===
if user_path == "":
    if is_colab():
        path = upload_via_colab()
    else:
        raise FileNotFoundError(
            "Nebyla zadána cesta a neběžíš v Google Colab.\n"
            "Prosím zadej cestu k .csv/.xlsx souboru do inputu a spusť znovu."
        )
else:
    path = normalize_path(user_path)
    if not os.path.isfile(path):
        # Pokud je to složka, hledej uvnitř
        if os.path.isdir(path):
            cand = glob.glob(os.path.join(path, "*.csv")) + glob.glob(os.path.join(path, "*.xlsx"))
            if not cand:
                raise FileNotFoundError("Ve zadané složce jsem nenašel žádné .csv/.xlsx soubory.")
            cand.sort()
            path = cand[0]
            print(f"[INFO] Ve složce nalezen soubor: {os.path.basename(path)}")
        else:
            # Zkus najít podobný soubor
            alt = try_find_similar(path)
            if alt and os.path.isfile(alt):
                print(f"[INFO] Opravuji cestu na: {alt}")
                path = alt
            else:
                # Poslední pomoc: vypiš obsah složky, pokud existuje její rodič
                folder = os.path.dirname(path)
                if os.path.isdir(folder):
                    print(f"[DEBUG] Obsah složky {folder}:")
                    for f in os.listdir(folder):
                        print(" -", f)
                raise FileNotFoundError(f"Soubor {path!r} neexistuje.")

# === Načtení dat (CSV/Excel) ===
ext = os.path.splitext(path)[1].lower()
if ext == ".csv":
    try:
        data_breaches = pd.read_csv(path, sep=None, engine="python")
    except UnicodeDecodeError:
        data_breaches = pd.read_csv(path, sep=None, engine="python", encoding="utf-8", encoding_errors="ignore")
elif ext in (".xlsx", ".xls"):
    xls = pd.ExcelFile(path)
    # zvol list s nejvíce řádky (rychlý odhad)
    best_sheet = max(xls.sheet_names, key=lambda s: pd.read_excel(path, sheet_name=s, nrows=5).shape[0])
    data_breaches = pd.read_excel(path, sheet_name=best_sheet)
    print(f"[INFO] Excel sheet: {best_sheet}")
else:
    raise ValueError("Podporovány jsou pouze .csv a .xlsx/.xls")

orig_df = data_breaches.copy()

# === Profil + heuristiky konverzí ===
print(f"\n[INFO] Načten soubor: {path}")
print(f"[INFO] Tvar: {data_breaches.shape[0]} řádků × {data_breaches.shape[1]} sloupců")
print("[INFO] Dtypes:\n", data_breaches.dtypes)

date_like, numeric_like, changes = [], [], []
for col in data_breaches.columns:
    cl = col.lower()
    if re.search(r"(date|time|year)", cl): date_like.append(col)
    if re.search(r"(record|count|affected|size|num|amount|total)", cl): numeric_like.append(col)

# Datumy
for col in date_like:
    if col in data_breaches.columns:
        before = data_breaches[col].isna().sum()
        data_breaches[col] = pd.to_datetime(data_breaches[col], errors="coerce")
        after = data_breaches[col].isna().sum()
        changes.append(("to_datetime", col, f"NaN změna: {after - before}"))

# Čísla
def to_number(v):
    if pd.isna(v): return np.nan
    s = re.sub(r"[^\d\.\-]", "", str(v))
    if s in ("", "-", ".", "-.", ".-"): return np.nan
    try: return float(s)
    except: return np.nan

for col in numeric_like:
    if col in data_breaches.columns:
        before = data_breaches[col].notna().sum()
        data_breaches[col] = data_breaches[col].apply(to_number)
        after = data_breaches[col].notna().sum()
        changes.append(("to_numeric", col, f"ne-NaN: {before} → {after}"))

# Kategorie
cat_candidates = []
for col in data_breaches.select_dtypes(include=["object"]).columns:
    nun = data_breaches[col].nunique(dropna=True)
    if 1 < nun <= max(20, int(0.05 * len(data_breaches))):
        cat_candidates.append(col)
for col in cat_candidates:
    data_breaches[col] = data_breaches[col].astype("category")
    changes.append(("to_category", col, f"nunique={data_breaches[col].nunique()}"))

missing_summary = data_breaches.isna().sum().sort_values(ascending=False)

# Uložení vedle zdroje
out_dir = os.path.dirname(path) or "."
clean_path = os.path.join(out_dir, "data_breaches_clean.csv")
data_breaches.to_csv(clean_path, index=False)

# === SOUHRNNÝ VÝPIS ===
print("\n================ SOUHRNNÝ VÝPIS =================")
print("co všechno bylo provedeno:")
print(f"- Soubor načten z: {path}")
print(f"- Původní tvar: {orig_df.shape[0]} řádků × {orig_df.shape[1]} sloupců")
print(f"- Aktuální tvar: {data_breaches.shape[0]} řádků × {data_breaches.shape[1]} sloupců")
print(f"- Kandidáti na datum: {date_like}")
print(f"- Kandidáti na čísla: {numeric_like}")
print(f"- Kandidáti na kategorie: {cat_candidates}")
print("- Provedené konverze/akce:")
for k, c, note in changes:
    print(f"  • {k:>12} :: {c} — {note}")
print("\n[INFO] Top 15 sloupců s chybějícími hodnotami:")
print(missing_summary.head(15))
print(f"\n[OK] Očištěný soubor uložen: {clean_path}")


Zadej úplnou nebo relativní cestu k souboru (.csv / .xlsx).
Příklad:  C:/Users/ja/Downloads/data_breaches.csv  nebo  ./Data_Breaches/data_breaches.xlsx


Krátká interpretace

Buňka nejdřív vyžádá cestu k souboru přes input().

Pokud necháš prázdné a jsi v Google Colab, otevře se upload dialog.

Poté se provede profil dat, konverze datumu/čísel/kategorií, výpis NaN a uloží se data_breaches_clean.csv do stejné složky, kde je zdroj.

Na konci je náš povinný souhrnný výpis „co všechno bylo provedeno“.